In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm     # pip3/pip/conda install tqdm
import os

In [ ]:
att_img = "train/0075.bmp"
att_img = cv2.imread(att_img, cv2.IMREAD_GRAYSCALE)
padding = 100
att_img = np.pad(att_img, ((padding, padding), (padding, padding)), 'constant', constant_values=(0, 0))

plt.imshow(att_img, cmap='gray')

In [ ]:
# 滤波前是否需要中值滤波？
blurred_img = cv2.medianBlur(att_img, 5)

_, thres_img = cv2.threshold(blurred_img, 80, 255, cv2.THRESH_BINARY)
plt.imshow(thres_img, cmap='gray')

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10)) # 参数是否可以自适应
morphed_img = cv2.morphologyEx(thres_img, cv2.MORPH_CLOSE, kernel)

plt.imshow(morphed_img, cmap='gray')

In [ ]:
# https://www.jb51.net/article/164348.htm
canny = cv2.Canny(morphed_img, 40, 150)

contour_img = cv2.cvtColor(att_img, cv2.COLOR_GRAY2BGR)
contours, _ = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour_img = cv2.drawContours(contour_img, contours, -1, (255, 0, 0), 3)

plt.imshow(contour_img)

In [ ]:
def solve(p1, p2, p3, p4):
    A = np.array([[p2[1]-p1[1], p1[0]-p2[0]], [p4[1]-p3[1], p3[0]-p4[0]]])
    B1 = np.array([[(p2[1]-p1[1])*p1[0]+(p1[0]-p2[0])*p1[1], p1[0]-p2[0]], 
                  [(p4[1]-p3[1])*p3[0]+(p3[0]-p4[0])*p3[1], p3[0]-p4[0]]])
    B2 = np.array([[p2[1]-p1[1], (p2[1]-p1[1])*p1[0]+(p1[0]-p2[0])*p1[1]],
                  [p4[1]-p3[1], (p4[1]-p3[1])*p3[0]+(p3[0]-p4[0])*p3[1]]])
    A, B1, B2 = map(lambda x: np.linalg.det(x), (A, B1, B2))
    return B1/A, B2/A

def handle_not_rect(points):
    new_points = np.zeros((points.shape[0] - 1, points.shape[1]))
    dist_list = [np.linalg.norm(points[i] - points[i - 1]) for i in range(points.shape[0])]
    i = np.argmin(np.array(dist_list)) # 缩点(i-2, i-1), (i, i+1)
    x, y = solve(points[i-2], points[i-1], points[i], points[(i+1)%points.shape[0]])
    # print(new_points[:i-1, :].shape, points[:i-1, :].shape, i)
    if i > 0:
        new_points[:i-1, :] = points[:i-1, :]
    new_points[i-1, :] = np.array([x, y])
    new_points[i:, :] = points[i+1:, :]
    return new_points


In [ ]:
max_contour ,max_area = 0, 0
for contour in contours:
    tmp = cv2.contourArea(contour)
    if tmp > max_area:
        max_contour, max_area = contour, tmp
print(max_contour.shape)
max_contour = max_contour[::-1, :, :]

# robustness：如果不是4怎么办   0.02来自于网络
poly_contour = cv2.approxPolyDP(max_contour, 0.02 * cv2.arcLength(max_contour, True), True)
# assert(len(poly_contour) == 4)
print(poly_contour)

shape_x, shape_y = int(856 / 1.5), int(540 / 1.5) # 蓝票标准长宽比
points = poly_contour.squeeze()
while points.shape[0] > 4:
    points = handle_not_rect(points)
print(points)

contour_img = cv2.cvtColor(att_img, cv2.COLOR_GRAY2BGR)
contour_img = cv2.drawContours(contour_img, [poly_contour], -1, (255, 0, 0), 3)
plt.imshow(contour_img)

In [ ]:
c = np.cross(points[1]-points[0], points[2]-points[1])
if c > 0:   # 修正为右旋标价
    points = points[::-1, :]

len1 = np.linalg.norm(points[0] - points[1])
len2 = np.linalg.norm(points[1] - points[2])
if len1 >= len2:    # 短边优先
    points = np.roll(points, 1, axis=0)

if points[0][0] > points[2][0]: # y值小的优先
    points = np.roll(points, 2, axis=0)
print(points, poly_contour.shape)

poly_contour2 = points.reshape((4, 1, 2)).astype(np.int32)
contour_img = cv2.cvtColor(att_img, cv2.COLOR_GRAY2BGR)
contour_img = cv2.drawContours(contour_img, [poly_contour2], -1, (255, 0, 0), 3)
plt.imshow(contour_img)


In [ ]:
outer_x, outer_y = 20, 20

N = np.array([[outer_x, outer_y], [outer_x, shape_y - outer_y], 
            [shape_x - outer_x, shape_y - outer_y], [shape_x - outer_x, outer_y]])
mat = cv2.getPerspectiveTransform(points.astype(np.float32), N.astype(np.float32))
output1 = cv2.warpPerspective(att_img, mat, (shape_x, shape_y))
plt.imshow(output1, cmap='gray')



最后用霍夫变换fix一下

In [ ]:
blurred_img = cv2.medianBlur(output1, 5)
_, thres_img = cv2.threshold(blurred_img, 80, 255, cv2.THRESH_BINARY)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
morphed_img = cv2.morphologyEx(thres_img, cv2.MORPH_CLOSE, kernel)

canny = cv2.Canny(morphed_img, 40, 150)
plt.imshow(canny, cmap='gray')

In [ ]:
def rotate_if_reversed(input_img):
    img = cv2.medianBlur(input_img, 5)
    _, img = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY)

    r, c = img.shape
    lu = img[outer_y>>1: int(shape_y*0.4), outer_x>>1: int(shape_x*0.25)]
    rd = img[r-int(shape_y*0.4):r-(outer_y>>1), c-int(shape_x*0.25): c-(outer_x>>1)]
    if(np.mean(lu) < np.mean(rd)):
        input_img = cv2.flip(input_img, -1)
    return input_img

In [ ]:
lines = cv2.HoughLines(canny, 1, np.pi / 180, 100)
# print(lines)
lines = lines.squeeze()

angle, cnt = 0, 0
for _, theta in lines:
    theta = theta * 180 / np.pi - 90
    if np.abs(theta) < 10:
        angle = angle * cnt + theta
        cnt += 1
        angle /= cnt
mat = cv2.getRotationMatrix2D((shape_x/2, shape_y/2), angle, 1)
fixed_img = cv2.warpAffine(output1, mat, (shape_x, shape_y))

# 裁剪一下边缘
output = fixed_img[outer_y>>1:shape_y-(outer_y>>1), outer_x>>1:shape_x-(outer_x>>1)]
output = rotate_if_reversed(output)

plt.imshow(output, cmap='gray')

以下是上述操作的汇总

In [ ]:
def extract_ticket(att_img):
    padding = 100
    att_img = np.pad(att_img, ((padding, padding), (padding, padding)), 'constant', constant_values=(0, 0))
    blurred_img = cv2.medianBlur(att_img, 5)
    _, thres_img = cv2.threshold(blurred_img, 80, 255, cv2.THRESH_BINARY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    morphed_img = cv2.morphologyEx(thres_img, cv2.MORPH_CLOSE, kernel)

    canny = cv2.Canny(morphed_img, 40, 150)
    contours, _ = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    max_contour ,max_area = 0, 0
    for contour in contours:
        tmp = cv2.contourArea(contour)
        if tmp > max_area:
            max_contour, max_area = contour, tmp

    poly_contour = cv2.approxPolyDP(max_contour, 0.02 * cv2.arcLength(max_contour, True), True)
    points = poly_contour.squeeze()
    while points.shape[0] > 4:
        points = handle_not_rect(points)

    shape_x, shape_y = int(856 / 1.5), int(540 / 1.5) # 蓝票标准长宽比

    c = np.cross(points[1]-points[0], points[2]-points[1])
    if c > 0:   # 修正为右旋标价
        points = points[::-1, :]

    len1 = np.linalg.norm(points[0] - points[1])
    len2 = np.linalg.norm(points[1] - points[2])
    if len1 >= len2:    # 短边优先
        points = np.roll(points, 1, axis=0)

    if points[0][0] > points[2][0]: # y值小的优先
        points = np.roll(points, 2, axis=0)
    
    outer_x, outer_y = 20, 20

    N = np.array([[outer_x, outer_y], [outer_x, shape_y - outer_y], 
                [shape_x - outer_x, shape_y - outer_y], [shape_x - outer_x, outer_y]])
    mat = cv2.getPerspectiveTransform(points.astype(np.float32), N.astype(np.float32))
    return cv2.warpPerspective(att_img, mat, (shape_x, shape_y))

In [ ]:
def hoffle_fix(output1):
    blurred_img = cv2.medianBlur(output1, 5)
    _, thres_img = cv2.threshold(blurred_img, 80, 255, cv2.THRESH_BINARY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    morphed_img = cv2.morphologyEx(thres_img, cv2.MORPH_CLOSE, kernel)

    canny = cv2.Canny(morphed_img, 40, 150)

    lines = cv2.HoughLines(canny, 1, np.pi / 180, 100)
    lines = lines.squeeze()

    angle, cnt = 0, 0
    for _, theta in lines:
        theta = theta * 180 / np.pi - 90
        if np.abs(theta) < 10:
            angle = angle * cnt + theta
            cnt += 1
            angle /= cnt
    mat = cv2.getRotationMatrix2D((shape_x/2, shape_y/2), angle, 1)
    fixed_img = cv2.warpAffine(output1, mat, (shape_x, shape_y))

    # 裁剪一下边缘
    return fixed_img[outer_y>>1:shape_y-(outer_y>>1), outer_x>>1:shape_x-(outer_x>>1)]

在所有训练集上跑：

In [ ]:
input_path = "./train"
output_path = "./tickets"

if not os.path.exists(output_path):
    os.mkdir(output_path)

for file_name in tqdm(os.listdir(input_path)):
    try:
        img = cv2.imread(os.path.join(input_path, file_name), cv2.IMREAD_GRAYSCALE)
        ticket_img = extract_ticket(img)
        ticket_img = hoffle_fix(ticket_img)
        ticket_img = rotate_if_reversed(ticket_img)
        cv2.imwrite(os.path.join(output_path, file_name), ticket_img)
    except RuntimeError as e:
        print("\nFail at {}: {}, skipped".format(file_name, e.args[0]))
print('\ndone!')